In [ ]:
# Import necessary modules and libraries
import socket # For TCP networking
import threading # For serving two clients simultaneously
import sys # For adding command line arguments
import time # For the sleep() function


# Constants
SERVER_ADDRESS = 'localhost' # Server to be connected address
SERVER_PORT = int(sys.argv[1]) # Server to be connected port


# Global variables
game_state = [' ', ' ', ' ',    # State of the game
              ' ', ' ', ' ',    # (last int is counting turns)
              ' ', ' ', ' ', 0]
result = 0 # 0 = No winner, 1 = P1 winner, 2 = P2 winner, 3 = Tie


# Functions
def recvall(connection):
    # Returns all available data from connection as str
    data = b''
    while True:
        temp = connection.recv(4096)
        data += temp
        if len(temp) < 4096:
            break
    data = data.decode()
    return data

def valid_move(game_state, move):
    # Returns true if move can be played, 
    # returns false otherwise.
    # move is directly from socket (str)
    
    try:
        move = int(move)
    except:
        return False
    
    possible_moves = [1, 2, 3, 
                      4, 5, 6, 
                      7, 8, 9]
    
    if (move in possible_moves) and game_state[move - 1] == ' ':
        # If move is in 1 - 9 and the play square is empty
        return True
    else:
        return False

def win_condition(game_state):
    # Returns False if no winner
    # Returns True if winner
    
    # Column
    if game_state[0] == game_state[3] == game_state[6] != ' ':
        return True
    elif game_state[1] == game_state[4] == game_state[7] != ' ':
        return True
    elif game_state[2] == game_state[5] == game_state[8] != ' ':
        return True
    
    # Row
    elif game_state[0] == game_state[1] == game_state[2] != ' ':
        return True
    elif game_state[3] == game_state[4] == game_state[5] != ' ':
        return True
    elif game_state[6] == game_state[7] == game_state[8] != ' ':
        return True
    
    # Crosses
    elif game_state[0] == game_state[4] == game_state[8] != ' ':
        return True
    elif game_state[2] == game_state[4] == game_state[6] != ' ':
        return True
    
    # No winner
    else:
        return False

def thread_function(player_id, connection):
    # Global variables
    global game_state
    global result
    # Assign player symbol
    if player_id == 1:
        player_symbol = 'X'
    elif player_id == 2:
        player_symbol = 'O'
    else:
        print('----- Error assigning player_symbol!')
        return
    # Explain the game
    connection.sendall('\n\n---------- Welcome to Tic Tac Toe! ----------\n\n'\
                       '- Your symbol is {}.\n\n'\
                       '- When it is your turn, use numbers 1-9 to make your move.\n\n'\
                       '- Submit anything when it is not your move to get game state\n\n'\
                       '- The play area is numbered as seen below:\n'\
                       '1|2|3\n'\
                       '-----\n'\
                       '4|5|6\n'\
                       '-----\n'\
                       '7|8|9\n'\
                       '-----'.format(player_symbol).encode())
    # Main loop
    while result == 0: # While the game is still going
        # Inform about turn, play area
        if (game_state[9] % 2) == 0: # P1 turn
            turn = '1'
        elif (game_state[9] % 2) == 1: # P2 turn
            turn = '2'
        else:
            print('----- Error with player_id while calculating turns!')
        if int(turn) == player_id:
            connection.sendall('\n\nYOUR TURN!'.encode())
        connection.sendall("\n\nPlayer {}'s turn!\n"\
                           "Current state of the game:\n"\
                           '{}|{}|{}\n'\
                           '-----\n'\
                           '{}|{}|{}\n'\
                           '-----\n'\
                           '{}|{}|{}\n'\
                           '-----'.format(turn, 
                                          game_state[0], game_state[1], 
                                          game_state[2], game_state[3], 
                                          game_state[4], game_state[5], 
                                          game_state[6], game_state[7], 
                                          game_state[8]).encode())
        # Wait for transmission
        move = recvall(connection)
        if move == 'NOP':
            continue
        # Respond and change parameters accordingly
        if player_id == 1: # P1 client
            if (game_state[9] % 2) == 0: # P1 turn
                # Check if valid move
                if valid_move(game_state, move) == False:
                    connection.sendall('\n\nInvalid move! Please try again.'.encode())
                    print('Player 1 invalid move.')
                    continue
                # Valid move, update game state
                game_state[int(move) - 1] = 'X'
                game_state[9] += 1
                print('Player 1 valid move on {}.'.format(str(move)))
                # Updated game state, check for winner
                if win_condition(game_state) == True:
                    result = 1
            else: # P2 turn
                # Wait your turn!
                continue
        elif player_id == 2: # P2 client
            if (game_state[9] % 2) == 1: # P2 turn
                # Check if valid move
                if valid_move(game_state, move) == False:
                    connection.sendall('\n\nInvalid move! Please try again.'.encode())
                    print('Player 2 invalid move.')
                    continue
                # Valid move, update game state
                game_state[int(move) - 1] = 'O'
                game_state[9] += 1
                print('Player 2 valid move on {}.'.format(str(move)))
                # Updated game state, check for winner
                if win_condition(game_state) == True:
                    result = 2
            else: # P1 turn
                # Wait your turn!
                continue
        else:
            print('----- Error with player_id!')
        # Check for tie
        if game_state[9] == 9 and win_condition(game_state) == False:
            result = 3
    # Game has concluded, result != 0
    if result == 3:
        connection.sendall('\n\nGame over. TIE!'.encode())
        print('Game over! Tie.')
    elif result == player_id:
        connection.sendall('\n\nGame over. YOU WIN!'.encode())
    else:
        connection.sendall('\n\nGame over. YOU LOSE!'.encode())
        print('Game over! Player {} wins.'.format(str(result)))


# ----- START -----
# Create socket and accept connections
socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
socket.bind((SERVER_ADDRESS, SERVER_PORT))
socket.listen(100)
connection_P1, address_P1 = socket.accept()
connection_P1.sendall('You are player 1 (P1). '\
                      'Please wait for player 2 to join.'.encode())
print('Player 1 has connected, their symbol is X.')
socket.listen(100)
connection_P2, address_P2 = socket.accept()
connection_P2.sendall('You are player 2 (P2).'.encode())
print('Player 2 has connected, their symbol is O.')
print('The game has started!')
# Create threads for both players
P1_thread = threading.Thread(target = thread_function, 
                             args = (1, connection_P1))
P2_thread = threading.Thread(target = thread_function, 
                             args = (2, connection_P2))
P1_thread.start()
P2_thread.start()
P1_thread.join()
P2_thread.join()